In [7]:

import torch
import pandas as pd
import numpy as np
import boto3
import numpy as np
import pandas as pd
from utils import get_args_parser
from remasker_impute import ReMasker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/Labitemlist.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
Labitemlist = pd.read_csv(obj['Body'], header=None,names=['labitem', 'cnt'], dtype=int)

file_path = 'Labrado/alllabs1000adm/unique_adm_year_day.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
unique_adm_year_day = pd.read_csv(obj['Body'], header=None,names=['hadm_id', 'chartyear', 'day','rowinx'], dtype=int)

unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

split year is 2178


In [8]:

file_path = 'Labrado/alllabs1000adm/npval.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval = pd.read_csv(obj['Body'], header=None, dtype=float)


file_path = 'Labrado/alllabs1000adm/npval_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval_last = pd.read_csv(obj['Body'], header=None, dtype=float)

In [18]:
file_path = 'Labrado/alllabs1000adm/nptime.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime = pd.read_csv(obj['Body'], header=None, dtype=float)

file_path = 'Labrado/alllabs1000adm/nptime_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime_last = pd.read_csv(obj['Body'], header=None, dtype=float)

KeyboardInterrupt: 

In [19]:
X_train[:,0:LanCnt*2:2].shape

(1582939, 10)

In [21]:
LanCnt = 10
X_train = np.full((len(Train_inx),LanCnt*4), np.nan)
X_train[:,0:LanCnt*2:2] = npval.iloc[Train_inx,:LanCnt]
X_train[:,1:LanCnt*2:2] = nptime.iloc[Train_inx,:LanCnt]
X_train[:,LanCnt*2:LanCnt*4:2] = npval_last.iloc[Train_inx,:LanCnt]
X_train[:,LanCnt*2+1:LanCnt*4:2] = nptime_last.iloc[Train_inx,:LanCnt]

X_test = np.full((len(Test_inx),LanCnt*4), np.nan)
X_test[:,0:LanCnt*2:2] = npval.iloc[Test_inx,:LanCnt]
X_test[:,1:LanCnt*2:2] = nptime.iloc[Test_inx,:LanCnt]
X_test[:,LanCnt*2:LanCnt*4:2] = npval_last.iloc[Test_inx,:LanCnt]
X_test[:,LanCnt*2+1:LanCnt*4:2] = nptime_last.iloc[Test_inx,:LanCnt]


In [27]:
X_train[:5,:]

array([[  3.5,  18. , 137. ,  18. , 103. ,  18. ,  26.5,  18. ,   1.1,
         18. ,  24. ,  18. ,  14. ,  18. ,  19. ,  18. , 169. ,  18. ,
        183. ,  18. ,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan],
       [  3.7,  18. , 136. ,  18. , 100. ,  18. ,  28.1,  18. ,   1.2,
         18. ,  28. ,  18. ,  12. ,  18. ,  17. ,  18. , 190. ,  18. ,
        195. ,  18. ,   3.5,  42. , 137. ,  42. , 103. ,  42. ,  26.5,
         42. ,   1.1,  42. ,  24. ,  42. ,  14. ,  42. ,  19. ,  42. ,
        169. ,  42. , 183. ,  42. ],
       [  3.9,  17. , 138. ,  17. , 102. ,  17. ,  24.4,  14. ,   0.9,
         17. ,  26. ,  17. ,  14. ,  17. ,  14. ,  17. , 177. ,  17. ,
        188. ,  14. ,   3.7,  42. , 136. ,  42. , 100. ,  42. ,  28.1,
         42. ,   1.2,  42. ,  28. ,  42. ,  12. ,  42. ,  17. ,  42. ,
        190. ,  42. , 195. ,  42. ],
       [  5.2,  17. , 140. ,  17. , 1

In [ ]:
np.savetxt('X_train.csv', X_train, delimiter=',')
np.savetxt('X_test.csv', X_test, delimiter=',')